In [1]:
json_input = {
  "drawflow": {
    "Home": {
      "data": {
        "29": {
          "id": 29,
          "name": "globalVariable",
          "data": {
            "id": 29,
            "name": "globalVariable",
            "data": {
              "id": 29,
              "name": "globalVariable",
              "data": {
                "id": 29,
                "name": "globalVariable",
                "data": {
                  "id": 29,
                  "name": "globalVariable",
                  "data": {
                    "key": "prompt_2",
                    "valueHolder": "Testing we are testing",
                    "method": "string"
                  },
                  "class": "globalVariable",
                  "html": "globalVariable",
                  "typenode": "vue",
                  "inputs": {},
                  "outputs": {},
                  "pos_x": 1872.5161743164053,
                  "pos_y": -224.87152099609366,
                  "key": "system_prompt",
                  "valueholder": "Testing we are testing",
                  "valueHolder": "Testing we are testing"
                },
                "class": "globalVariable",
                "html": "globalVariable",
                "typenode": "vue",
                "inputs": {},
                "outputs": {},
                "pos_x": 1391,
                "pos_y": -629,
                "key": "prompt_2",
                "valueHolder": "Testing we are testing"
              },
              "class": "globalVariable",
              "html": "globalVariable",
              "typenode": "vue",
              "inputs": {},
              "outputs": {},
              "pos_x": 1259,
              "pos_y": 720,
              "key": "prompt_2",
              "valueHolder": "Testing we are testing"
            },
            "class": "globalVariable",
            "html": "globalVariable",
            "typenode": "vue",
            "inputs": {},
            "outputs": {},
            "pos_x": 1259,
            "pos_y": 720
          },
          "class": "globalVariable",
          "html": "globalVariable",
          "typenode": "vue",
          "inputs": {},
          "outputs": {},
          "pos_x": 1259,
          "pos_y": 720
        },
        "65": {
          "id": 65,
          "name": "Node2",
          "data": {
            "id": 65,
            "name": "Node2",
            "data": {
              "script": "def handle_classified(user_message, sender_id, senderId_info):\n\n",
              "importStatements": "Import Statements\n\n"
            },
            "class": "Node2",
            "html": "Node2",
            "typenode": "vue",
            "inputs": {
              "input_1": {
                "connections": []
              }
            },
            "outputs": {
              "output_1": {
                "connections": []
              },
              "output_2": {
                "connections": []
              }
            },
            "pos_x": 1258.5,
            "pos_y": 554
          },
          "class": "Node2",
          "html": "Node2",
          "typenode": "vue",
          "inputs": {
            "input_1": {
              "connections": []
            }
          },
          "outputs": {
            "output_1": {
              "connections": []
            },
            "output_2": {
              "connections": []
            }
          },
          "pos_x": 1256,
          "pos_y": 535
        }
      }
    }
  }
}

In [2]:
def create_node_system(json_data):
    nodes = json_data['drawflow']['Home']['data']
    node_system = {}

    for node_id, node_info in nodes.items():
        # Initialize the node system dictionary for each node
        # Ensure internalData points to the innermost 'data' dictionary if it exists
        internal_data = node_info['data']['data'] if 'data' in node_info['data'] else {}
        node_system[node_id] = {
            "type": node_info['class'],
            "headNodeId": [],
            "childNodeId": [],
            "internalData": internal_data
        }

    # Populate headNodeId and childNodeId based on connections
    for node_id, node_info in nodes.items():
        # Handle outputs to determine child nodes
        if 'outputs' in node_info:
            for output_key, output_value in node_info['outputs'].items():
                for connection in output_value['connections']:
                    child_node_id = connection['node']
                    node_system[node_id]['childNodeId'].append(child_node_id)
                    # Also update the head node information for the connected node
                    if node_id not in node_system[child_node_id]['headNodeId']:
                        node_system[child_node_id]['headNodeId'].append(node_id)

    return node_system

# Example usage:
node_system_output = create_node_system(json_input)
print(node_system_output)

{'29': {'type': 'globalVariable', 'headNodeId': [], 'childNodeId': [], 'internalData': {'id': 29, 'name': 'globalVariable', 'data': {'id': 29, 'name': 'globalVariable', 'data': {'id': 29, 'name': 'globalVariable', 'data': {'key': 'prompt_2', 'valueHolder': 'Testing we are testing', 'method': 'string'}, 'class': 'globalVariable', 'html': 'globalVariable', 'typenode': 'vue', 'inputs': {}, 'outputs': {}, 'pos_x': 1872.5161743164053, 'pos_y': -224.87152099609366, 'key': 'system_prompt', 'valueholder': 'Testing we are testing', 'valueHolder': 'Testing we are testing'}, 'class': 'globalVariable', 'html': 'globalVariable', 'typenode': 'vue', 'inputs': {}, 'outputs': {}, 'pos_x': 1391, 'pos_y': -629, 'key': 'prompt_2', 'valueHolder': 'Testing we are testing'}, 'class': 'globalVariable', 'html': 'globalVariable', 'typenode': 'vue', 'inputs': {}, 'outputs': {}, 'pos_x': 1259, 'pos_y': 720, 'key': 'prompt_2', 'valueHolder': 'Testing we are testing'}}, '65': {'type': 'Node2', 'headNodeId': [], 'ch

In [15]:
import json

def find_deepest_dict(data):
    deepest_dict = None
    max_depth = 0

    def recurse(item, depth):
        nonlocal deepest_dict, max_depth
        if isinstance(item, dict):
            if depth > max_depth:
                max_depth = depth
                deepest_dict = item
            for value in item.values():
                recurse(value, depth + 1)

    recurse(data, 1)
    return deepest_dict

def generate_code(node_system):
    code = []  # Initialize code as an empty list
    global_variables = {}

    # print("node_system.items()", node_system.items())

    # Extract global variables
    for node_id, node_data in node_system.items():
        if node_data['type'] == 'globalVariable':
            deepest_data = find_deepest_dict(node_data['internalData'])
            if deepest_data:  # Ensure deepest data was found
                key = deepest_data['key']
                value = deepest_data['valueHolder']
                method = deepest_data['method']
                global_variables[key] = (value, method)

    # Generate global variable assignments
    for key, (value, method) in global_variables.items():
        if method == 'string':
            code.append(f"{key} = '{value}'")
        else:
            code.append(f"{key} = {value}")

    # Find the starting node
    start_node_id = None
    for node_id, node_data in node_system.items():
        if node_data['type'] == 'fbWebhook' and not node_data['headNodeId']:
            start_node_id = node_id
            break

    # Generate code for each node
    visited_nodes = set()
    def generate_node_code(node_id, indent="", head_node_id=None):
        if node_id in visited_nodes:
            return
        visited_nodes.add(node_id)

        node_data = node_system[node_id]
        node_type = node_data['type']

        if node_type == 'fbWebhook':
            code_line = f"{indent}fbWebhook()"
            code_line = f"{indent}node{node_id} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, f"node{node_id}")

        elif node_type == 'llmGenerate':
            system_prompt = node_data['internalData']['systemPrompt']
            user_prompt = node_data['internalData']['userPrompt']
            code_line = f"{indent}llmGenerate({system_prompt}, {user_prompt})"
            code_line = f"{indent}node{node_id} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, f"node{node_id}")

        elif node_type == 'extractOutput':
            code_line = f"{indent}extractOutput({head_node_id})"
            code_line = f"{indent}node{node_id} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, f"node{node_id}")

        elif node_type == 'switch':
            internal_key = node_data['internalData']['input']
            code.append(f"{indent}match {head_node_id}['{internal_key}']:")
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent + "    ", head_node_id)

        elif node_type == 'switchCase':
            input_value = node_data['internalData']['input']
            code.append(f"{indent}case {input_value}:")
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent + "    ", head_node_id)

        elif node_type == 'variableSet':
            key = node_data['internalData']['key']
            code.append(f"{indent}{key} = {head_node_id}")
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, key)
        
        elif node_type == 'handleUser':
            key = node_data['internalData']['key']
            code_line = f"{indent}handleUser({head_node_id})"
            code_line = f"{indent}{key} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, key)

        elif node_type == 'functionWrapper':
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, f"node{node_data['headNodeId'][0]}")

    generate_node_code(start_node_id)
    return "\n".join(code)

# Assuming 'node_system_output' is a properly structured dictionary as needed by the function
# Example usage
generated_code = generate_code(node_system_output)
print(generated_code)


prompt_2 = 'Testing we are testing'
prompt_3 = 'We are testing'
node49 = fbWebhook()
incoming_message = node49
users_info = handleUser(incoming_message)
match users_info['send_message']:
    case True:
        node56 =         llmGenerate(system_prompt, incoming_message)
        node58 =         extractOutput(node56)
        llm_generated = node58
        match llm_generated['type']:
            case text:
            case handle_assistant:
            case function:
    case False:


In [9]:
import json

def find_deepest_dict(data):
    deepest_dict = None
    max_depth = 0

    def recurse(item, depth):
        nonlocal deepest_dict, max_depth
        if isinstance(item, dict):
            if depth > max_depth:
                max_depth = depth
                deepest_dict = item
            for value in item.values():
                recurse(value, depth + 1)

    recurse(data, 1)
    return deepest_dict

def generate_code(node_system):
    code = []  # Initialize code as an empty list
    global_variables = {}

    # Extract global variables
    for node_id, node_data in node_system.items():
        if node_data['type'] == 'globalVariable':
            deepest_data = find_deepest_dict(node_data['internalData'])
            if deepest_data:  # Ensure deepest data was found
                key = deepest_data['key']
                value = deepest_data['valueHolder']
                method = deepest_data['method']
                global_variables[key] = (value, method)

    # Generate global variable assignments
    for key, (value, method) in global_variables.items():
        if method == 'string':
            code.append(f"{key} = '{value}'")
        else:
            code.append(f"{key} = {value}")

    # Find the starting node
    start_node_id = None
    for node_id, node_data in node_system.items():
        if node_data['type'] == 'fbWebhook' and not node_data['headNodeId']:
            start_node_id = node_id
            break

    # Generate code for each node
    visited_nodes = set()
    def generate_node_code(node_id, indent="", head_node_id=None):
        if node_id in visited_nodes:
            return
        visited_nodes.add(node_id)

        node_data = node_system[node_id]
        node_type = node_data['type']

        if node_type == 'fbWebhook':
            code_line = f"{indent}fbWebhook()"
            code_line = f"{indent}node{node_id} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, f"node{node_id}")

        elif node_type == 'llmGenerate':
            system_prompt = node_data['internalData']['systemPrompt']
            user_prompt = node_data['internalData']['userPrompt']
            code_line = f"{indent}llmGenerate({system_prompt}, {user_prompt})"
            code_line = f"{indent}node{node_id} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, f"node{node_id}")

        elif node_type == 'extractOutput':
            code_line = f"{indent}extractOutput({head_node_id})"
            code_line = f"{indent}node{node_id} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, f"node{node_id}")

        elif node_type == 'switch':
            code.append(f"{indent}match {head_node_id}:")
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent + "    ", head_node_id)

        elif node_type == 'switchCase':
            input_value = node_data['internalData']['input']
            code.append(f"{indent}case '{input_value}':")
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent + "    ", head_node_id)

        elif node_type == 'variableSet':
            key = node_data['internalData']['key']
            code.append(f"{indent}{key} = {head_node_id}")
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, key)

        elif node_type == 'functionWrapper':
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, f"node{node_data['headNodeId'][0]}")

        elif node_type == 'handleUser':
            key = node_data['internalData']['key']
            code_line = f"{indent}handleUser({head_node_id})"
            code_line = f"{indent}{key} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, key)

        elif node_type == 'handleUser':
            key = node_data['internalData']['key']
            code_line = f"{indent}handleUser({head_node_id})"
            code_line = f"{indent}{key} = " + code_line
            code.append(code_line)
            for child_node_id in node_data['childNodeId']:
                generate_node_code(child_node_id, indent, key)

    generate_node_code(start_node_id)
    return "\n".join(code)


generated_code = generate_code(node_system_output)
print(generated_code)


KeyError: 'systemPrompt'